In [ ]:
# Import all the requiered dependencies
import numpy as np
import os
import sys
from tqdm import tqdm

# Graphic tools
import matplotlib.pyplot as plt

# Tools from Swarm Systems Lab simulator
from ssl_simulator import createDir
from ssl_simulator.math import uniform_distrib, elliptical_distrib
from ssl_simulator.math import gen_Z_random, R_2D_matrix
from ssl_simulator.visualization import set_paper_parameters

set_paper_parameters(fontsize=12)

# --------------------------------------------------------------------------------------

# Python project to path 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the main module of the project
from dfc import SimulatorDistrNew, plot_resilience_paper

# Output directory
OUTPUT_PATH = os.path.join("..", "output")
createDir(OUTPUT_PATH)

## Definition of C

In [ ]:
# ------ Definition of C plot



# Generate the distributions
xy1 = elliptical_distrib(100, , rx=1.0, ry=1.0, h=0.0, border_noise=0.0)
xy2 = 2
xy3 = 3

# Plot



## SIM 1 - Resilience simulation: robots disconnect during the mission

In [ ]:
# Generate the robots distribution and set the initial positions
N = 70
lims = [3, 1]
rc0 = [0, 0]

p0 = uniform_distrib(N=N, lims=lims, rc0=rc0, seed=2024)

# Generate the graph
Z = gen_Z_random(N, rounds=3, seed=2024)

# Generating the simulator class
dt = 0.001
lambda_d = [10, 4]

epsilon_phat = 1/4
epsilon_chat = 1/10

sim = SimulatorDistrNew(
    p0=p0, Z=Z, dt=dt, lambda_d=lambda_d, ke=0.8, kpc=1, kc=1, 
    epsilon_phat=epsilon_phat, epsilon_chat=epsilon_chat)

# Graph connectivity (lambda2)
print(Z)
print(r"$\lambda_0$:", sim.lambda_min)

In [ ]:
# ----------------------------------------------------------------------
# Execute the simulation given a desired final time stamp
# ----------------------------------------------------------------------
tf = 2

t_list = np.arange(0, tf, dt)
for it in tqdm(range(len(t_list))):

    # [!!] Killing robots may disconect the graph, yielding to numerical errors
    # in the simulation (it has to be fixed)

    if it > 0.3/dt:
        sim.kill_agent(np.arange(0,5,1))
    if it > 0.5/dt:
        sim.kill_agent(np.arange(5,10,1))
    if it > 1/dt:
        sim.kill_agent(np.arange(10,15,1))

    sim.int_euler()

In [ ]:
# Generate the plot
plot_resilience_paper(
    sim, limx=11, dpi=400, figsize=(12,5.5), t_sep=0.5,
    title=rf"n = {N}, $\varepsilon_\text f$ = {epsilon_chat}, $\varepsilon_\text s$ = {epsilon_phat} " +
    rf"$\qquad\qquad\qquad\qquad\qquad$ $\lambda_1^*$ = {lambda_d[0]}, $\lambda_2^*$ = {lambda_d[1]}")

## SIM 2 - Resilience simulation: rotating agents

In [ ]:
# Generate the robots distribution and set the initial positions
N = 70
lims = [3, 1]
rc0 = [0, 0]

p0 = uniform_distrib(N=N, lims=lims, rc0=rc0, seed=2024)

# Generate the graph
Z = gen_Z_random(N, rounds=3, seed=2024)

# Generating the simulator class
dt = 0.001
lambda_d = [10, 4]

epsilon_phat = 1/4
epsilon_chat = 1/10

sim = SimulatorDistrNew(
    p0=p0, Z=Z, dt=dt, lambda_d=lambda_d, ke=0.8, kpc=1, kc=1, 
    epsilon_phat=epsilon_phat, epsilon_chat=epsilon_chat)

# Graph connectivity (lambda2)
print(Z)
print(r"$\lambda_0$:", sim.lambda_min)

In [ ]:
# ----------------------------------------------------------------------
# Execute the simulation given a desired final time stamp
# ----------------------------------------------------------------------
tf = 2

R1 = R_2D_matrix(2*np.pi/3*2*dt)
R2 = R_2D_matrix(2*np.pi*dt)
R3 = R_2D_matrix(2*np.pi/2*dt)
R4 = R_2D_matrix(2*np.pi*2*dt)
sim.status_dyn[N-4:N] = 0

t_list = np.arange(0, tf, dt)
for it in tqdm(range(len(t_list))):
    sim.variables["p"][-1] = (R1 @ sim.variables["p"][-1][:,None]).squeeze()
    sim.variables["p"][-2] = (R2 @ sim.variables["p"][-2][:,None]).squeeze()
    sim.variables["p"][-3] = (R3 @ sim.variables["p"][-3][:,None]).squeeze()
    sim.variables["p"][-4] = (R4 @ sim.variables["p"][-4][:,None]).squeeze()
    sim.int_euler()

In [ ]:
# Generate the plot
plot_resilience_paper(
    sim, limx=11, dpi=400, figsize=(12,5.5), t_sep=0.5,
    title=rf"n = {N}, $\varepsilon_\text f$ = {epsilon_chat}, $\varepsilon_\text s$ = {epsilon_phat} " +
    rf"$\qquad\qquad\qquad\qquad\qquad$ $\lambda_1^*$ = {lambda_d[0]}, $\lambda_2^*$ = {lambda_d[1]}")